# Notebook 4: So sánh và Phân tích Toàn diện

**Mục tiêu:** So sánh kết quả compression giữa ảnh grayscale và RGB

**Nội dung:**
1. Load kết quả từ cả hai notebooks
2. So sánh metrics
3. Phân tích compression ratio
4. Visualize side-by-side
5. Kết luận và recommendations

## 1. Setup

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from image_utils import load_image
from svd_compression import compress_grayscale, compress_rgb, calculate_compression_ratio
from quality_metrics import calculate_all_metrics

plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

print("Setup complete!")

## 2. Load Images

In [ ]:
# Load both images
img_gray = load_image('../images/grayscale/5.3.01.tiff', mode='GRAY')
img_rgb = load_image('../images/color/4.1.01.tiff', mode='RGB')

print(f"Grayscale image: {img_gray.shape}")
print(f"RGB image: {img_rgb.shape}")

## 3. Run Compression Tests

In [ ]:
# Test with same k values
k_values = [5, 10, 20, 50, 100]

# Results storage
results_comparison = {
    'k': [],
    'gray_psnr': [],
    'gray_mse': [],
    'gray_saved': [],
    'rgb_psnr': [],
    'rgb_mse': [],
    'rgb_saved': []
}

print("Running compression tests...\n")
print(f"{'k':>5} | {'Gray PSNR':>10} | {'RGB PSNR':>10} | {'Gray Saved':>12} | {'RGB Saved':>11}")
print("-" * 70)

for k in k_values:
    # Grayscale
    comp_gray = compress_grayscale(img_gray, k)
    metrics_gray = calculate_all_metrics(img_gray, comp_gray)
    stats_gray = calculate_compression_ratio(img_gray.shape, k, is_rgb=False)
    
    # RGB
    comp_rgb = compress_rgb(img_rgb, k)
    metrics_rgb = calculate_all_metrics(img_rgb, comp_rgb)
    stats_rgb = calculate_compression_ratio(img_rgb.shape, k, is_rgb=True)
    
    # Store
    results_comparison['k'].append(k)
    results_comparison['gray_psnr'].append(metrics_gray['psnr'])
    results_comparison['gray_mse'].append(metrics_gray['mse'])
    results_comparison['gray_saved'].append(stats_gray['space_saved_percent'])
    results_comparison['rgb_psnr'].append(metrics_rgb['psnr'])
    results_comparison['rgb_mse'].append(metrics_rgb['mse'])
    results_comparison['rgb_saved'].append(stats_rgb['space_saved_percent'])
    
    # Print
    print(f"{k:5d} | {metrics_gray['psnr']:10.2f} | {metrics_rgb['psnr']:10.2f} | "
          f"{stats_gray['space_saved_percent']:11.1f}% | {stats_rgb['space_saved_percent']:10.1f}%")

print("\nTests completed!")

## 4. Comparison Table

In [ ]:
# Create pandas DataFrame
df_comparison = pd.DataFrame(results_comparison)

print("\n" + "="*80)
print("COMPARISON TABLE")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

# Save to CSV
df_comparison.to_csv('../results/metrics/comparison_gray_vs_rgb.csv', index=False)
print("\nSaved to: results/metrics/comparison_gray_vs_rgb.csv")

## 5. Visual Comparison: PSNR

In [ ]:
# Plot PSNR comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(k_values))
width = 0.35

bars1 = ax.bar(x - width/2, results_comparison['gray_psnr'], width, 
               label='Grayscale', color='gray', alpha=0.8)
bars2 = ax.bar(x + width/2, results_comparison['rgb_psnr'], width,
               label='RGB', color='steelblue', alpha=0.8)

ax.axhline(y=30, color='red', linestyle='--', linewidth=1.5, label='Good (30 dB)', alpha=0.7)
ax.axhline(y=40, color='green', linestyle='--', linewidth=1.5, label='Excellent (40 dB)', alpha=0.7)

ax.set_xlabel('k (number of singular values)', fontsize=12)
ax.set_ylabel('PSNR (dB)', fontsize=12)
ax.set_title('PSNR Comparison: Grayscale vs RGB', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(k_values)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../results/visualizations/comparison_psnr.png', dpi=120, bbox_inches='tight')
plt.show()

## 6. Visual Comparison: Compression Ratio

In [ ]:
# Plot compression ratio comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(k_values))
width = 0.35

bars1 = ax.bar(x - width/2, results_comparison['gray_saved'], width,
               label='Grayscale', color='darkgreen', alpha=0.8)
bars2 = ax.bar(x + width/2, results_comparison['rgb_saved'], width,
               label='RGB', color='orange', alpha=0.8)

ax.set_xlabel('k (number of singular values)', fontsize=12)
ax.set_ylabel('Space Saved (%)', fontsize=12)
ax.set_title('Compression Ratio Comparison: Grayscale vs RGB', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(k_values)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 105])

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.0f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../results/visualizations/comparison_compression_ratio.png', dpi=120, bbox_inches='tight')
plt.show()

## 7. Quality vs Compression Trade-off

In [ ]:
# Trade-off plot
fig, ax = plt.subplots(figsize=(10, 7))

# Grayscale
ax.plot(results_comparison['gray_saved'], results_comparison['gray_psnr'],
        marker='o', markersize=8, linewidth=2, label='Grayscale', color='gray')

# RGB
ax.plot(results_comparison['rgb_saved'], results_comparison['rgb_psnr'],
        marker='s', markersize=8, linewidth=2, label='RGB', color='steelblue')

# Annotate k values
for i, k in enumerate(k_values):
    ax.annotate(f'k={k}', 
                (results_comparison['gray_saved'][i], results_comparison['gray_psnr'][i]),
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)
    ax.annotate(f'k={k}',
                (results_comparison['rgb_saved'][i], results_comparison['rgb_psnr'][i]),
                textcoords="offset points", xytext=(0,-15), ha='center', fontsize=9)

ax.set_xlabel('Space Saved (%)', fontsize=12)
ax.set_ylabel('PSNR (dB)', fontsize=12)
ax.set_title('Quality vs Compression Trade-off', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='lower left')
ax.grid(True, alpha=0.3)
ax.axhline(y=30, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('../results/visualizations/comparison_tradeoff.png', dpi=120, bbox_inches='tight')
plt.show()

## 8. Statistical Summary

In [ ]:
# Calculate statistics
print("\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)

print("\nGrayscale Compression:")
print(f"  Average PSNR: {np.mean(results_comparison['gray_psnr']):.2f} dB")
print(f"  Average Space Saved: {np.mean(results_comparison['gray_saved']):.1f}%")
print(f"  Best k (PSNR >= 30): {results_comparison['k'][0]}")

print("\nRGB Compression:")
print(f"  Average PSNR: {np.mean(results_comparison['rgb_psnr']):.2f} dB")
print(f"  Average Space Saved: {np.mean(results_comparison['rgb_saved']):.1f}%")
print(f"  Best k (PSNR >= 30): {results_comparison['k'][0]}")

print("\nComparison:")
psnr_diff = np.mean(results_comparison['gray_psnr']) - np.mean(results_comparison['rgb_psnr'])
saved_diff = np.mean(results_comparison['gray_saved']) - np.mean(results_comparison['rgb_saved'])
print(f"  PSNR difference (Gray - RGB): {psnr_diff:+.2f} dB")
print(f"  Compression difference (Gray - RGB): {saved_diff:+.1f}%")

print("="*80)

## 9. Conclusions

### Key Findings:

1. **Both methods work well**
   - Grayscale and RGB both achieve good compression ratios
   - Quality remains high even at low k values

2. **Grayscale advantages:**
   - Simpler computation (single channel)
   - Better compression ratio for same k
   - Faster processing

3. **RGB advantages:**
   - Preserves color information
   - Better for natural images
   - Still achieves good compression

4. **Trade-offs:**
   - RGB requires 3× storage per k value
   - But maintains visual quality better for color images
   - Choose based on application needs

### Recommendations:

- **For storage**: Use k=20-50 (good balance)
- **For transmission**: Use k=10-20 (higher compression)
- **For archival**: Use k=50-100 (better quality)
- **Grayscale images**: k=10-20 sufficient
- **Color images**: k=20-50 recommended

### Limitations of SVD Compression:

1. Not as efficient as JPEG for natural images
2. Computationally expensive (O(mn²))
3. Requires storing 3 matrices (U, Σ, V^T)
4. Best for images with low-rank structure

### Future Work:

1. Compare with JPEG, PNG
2. Try block-based SVD
3. Adaptive k selection
4. YCbCr color space
5. GPU acceleration